## Goller

In [23]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import re


STATS_URL = "https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/goals/premier-league"
OUTPUT_CSV_FILE = "player_stats.csv"
PLAYERS_PER_PAGE = 10
PAGES_TO_SCRAPE = 3


options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument("log-level=3")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

print("Selenium WebDriver başlatıldı.")

all_players_data = []

def safe_int_extract(text):
    """Metinden sayısal değerleri güvenli bir şekilde çıkarır"""
    if not text:
        return 0
    
    
    numbers = re.findall(r'\d+', text.strip())
    if numbers:
        return int(numbers[0])
    return 0

def get_stat_value_by_label(driver, wait, stat_label):
    """Etiket adıyla istatistik değerini alır (css-170fd60-StatValue class'ından)"""
    try:
        
        label_xpath = f"//span[contains(@class, 'css-1m9s03m-StatTitle') and text()='{stat_label}']/parent::div/preceding-sibling::div[contains(@class, 'css-170fd60-StatValue')]//span"
        element = wait.until(EC.visibility_of_element_located((By.XPATH, label_xpath)))
        value_text = element.text.strip()
        return safe_int_extract(value_text)
    
    except (TimeoutException, NoSuchElementException):
        
        try:
            
            stat_box_xpath = f"//div[contains(@class, 'css-8ko4sn-StatBox')]//span[contains(@class, 'css-1m9s03m-StatTitle') and text()='{stat_label}']/parent::div/preceding-sibling::div[contains(@class, 'css-170fd60-StatValue')]//span"
            element = driver.find_element(By.XPATH, stat_box_xpath)
            value_text = element.text.strip()
            return safe_int_extract(value_text)
        except:
            pass
        
        
        try:
            stat_boxes = driver.find_elements(By.CSS_SELECTOR, "div.css-8ko4sn-StatBox")
            
            for box in stat_boxes:
                try:
                    
                    title_element = box.find_element(By.CSS_SELECTOR, "span.css-1m9s03m-StatTitle")
                    if title_element.text.strip() == stat_label:
                        
                        value_element = box.find_element(By.CSS_SELECTOR, "div.css-170fd60-StatValue span")
                        return safe_int_extract(value_element.text)
                except:
                    continue
                    
        except Exception as e:
            print(f"    Son alternatif yöntem hatası: {e}")
        
        print(f"    Uyarı: {stat_label} değeri bulunamadı, 0 olarak ayarlandı")
        return 0

try:
    driver.get(STATS_URL)
    print(f"Hedef URL'ye gidiliyor: {STATS_URL}")
    
    wait = WebDriverWait(driver, 15)

    try:
        print("Çerez onay penceresi bekleniyor...")
        accept_cookies_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_cookies_button.click()
        print("Çerezler kabul edildi.")
        time.sleep(2)
    except TimeoutException:
        print("Çerez onay penceresi bulunamadı, devam ediliyor.")

    
    for page_num in range(PAGES_TO_SCRAPE):
        current_page = page_num + 1
        print(f"\n--- Sayfa {current_page} işleniyor ---")

        
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/tr/players/']")))
        
        player_rows = driver.find_elements(By.CSS_SELECTOR, "a[href*='/tr/players/']")
        player_urls_on_page = [row.get_attribute('href') for row in player_rows[:PLAYERS_PER_PAGE]]
        
        print(f"  Bu sayfada {len(player_urls_on_page)} oyuncu bulundu")
        
        for i, player_url in enumerate(player_urls_on_page):
            player_data = {}
            
            try:
                driver.get(player_url)
                
                rank = page_num * PLAYERS_PER_PAGE + i + 1
                print(f"  ({rank}) Oyuncu profili yükleniyor: {player_url.split('/')[-1]}")

                
                player_name_element = wait.until(EC.visibility_of_element_located((By.TAG_NAME, "h1")))
                player_data['Player'] = player_name_element.text.strip()

                
                try:
                    team_selectors = [
                        "div.css-14k6s2u-TeamCSS",
                        "div[class*='TeamCSS']",
                        "div[class*='team']"
                    ]
                    
                    team_name = None
                    for selector in team_selectors:
                        try:
                            team_element = driver.find_element(By.CSS_SELECTOR, selector)
                            team_name = team_element.text.strip()
                            if team_name:
                                break
                        except:
                            continue
                    
                    player_data['Team'] = team_name if team_name else "Bilinmeyen"
                        
                except Exception as e:
                    player_data['Team'] = "Bilinmeyen"
                    print(f"    Uyarı: Takım adı alınırken hata: {e}")
                
                
                time.sleep(2)
                
                
                player_data['Goals'] = get_stat_value_by_label(driver, wait, 'Goller')
                player_data['Matches'] = get_stat_value_by_label(driver, wait, 'Maçlar')

                print(f"  Veri çekildi: {player_data['Player']} | Takım: {player_data['Team']} | Maç: {player_data['Matches']} | Gol: {player_data['Goals']}")
                all_players_data.append(player_data)

            except Exception as e:
                print(f"  Hata: {player_url.split('/')[-1]} için veri çekilemedi. Hata: {e}")
                
                if 'Player' in player_data:
                    player_data.setdefault('Team', 'Bilinmeyen')
                    player_data.setdefault('Goals', 0)
                    player_data.setdefault('Matches', 0)
                    all_players_data.append(player_data)
            
            
            try:
                driver.back()
                
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/tr/players/']")))
                time.sleep(1)
            except Exception as e:
                print(f"  Geri dönüş hatası: {e}")
                
                driver.get(STATS_URL)
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/tr/players/']")))
                time.sleep(2)

        
        if page_num < PAGES_TO_SCRAPE - 1:
            print(f"\n{current_page}. sayfadan {current_page + 1}. sayfaya geçiliyor...")
            
            try:
                
                next_button_selectors = [
                    "//button[contains(text(), 'Sonraki')]",
                    "//button[contains(., 'Sonraki')]",
                    "//button[contains(@class, 'next')]",
                    "//button[contains(@aria-label, 'next')]",
                    "//button[contains(@aria-label, 'Sonraki')]"
                ]
                
                next_button = None
                for selector in next_button_selectors:
                    try:
                        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                        print(f"  'Sonraki' butonu bulundu: {selector}")
                        break
                    except TimeoutException:
                        continue
                
                if next_button:
                    
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    time.sleep(1)
                    
                  
                    try:
                        next_button.click()
                    except:
                        
                        driver.execute_script("arguments[0].click();", next_button)
                    
                    print(f"  Başarıyla {current_page + 1}. sayfaya geçildi.")
                    time.sleep(3)  
                    
                    
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/tr/players/']")))
                    
                else:
                    print("  'Sonraki' butonu bulunamadı. İşlem sonlandırılıyor.")
                    break

            except Exception as e:
                print(f"  Sonraki sayfaya geçerken hata: {e}")
                print("  İşlem sonlandırılıyor.")
                break

finally:
    driver.quit()
    print("\nSelenium WebDriver kapatıldı.")


if all_players_data:
    df = pd.DataFrame(all_players_data)
    
    
    df.insert(0, 'Rank', range(1, 1 + len(df)))
    
   
    df = df[['Rank', 'Player', 'Team', 'Matches', 'Goals']]
    
    
    df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')
    print(f"\nVeriler başarıyla '{OUTPUT_CSV_FILE}' dosyasına kaydedildi.")
    print(f"Toplam {len(df)} oyuncu verisi toplandı.")
    print("\n--- İlk 5 Satır ---")
    print(df.head())
    print("\n--- Son 5 Satır ---")
    print(df.tail())
else:
    print("\nHiçbir veri toplanamadığı için CSV dosyası oluşturulmadı.")

Selenium WebDriver başlatıldı.
Hedef URL'ye gidiliyor: https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/goals/premier-league
Çerez onay penceresi bekleniyor...
Çerez onay penceresi bulunamadı, devam ediliyor.

--- Sayfa 1 işleniyor ---
  Bu sayfada 10 oyuncu bulundu
  (1) Oyuncu profili yükleniyor: mohamed-salah
  Veri çekildi: Mohamed Salah | Takım: Liverpool | Maç: 38 | Gol: 29
  (2) Oyuncu profili yükleniyor: alexander-isak
  Veri çekildi: Alexander Isak | Takım: Newcastle United | Maç: 34 | Gol: 23
  (3) Oyuncu profili yükleniyor: erling-haaland
  Veri çekildi: Erling Haaland | Takım: Manchester City | Maç: 31 | Gol: 22
  (4) Oyuncu profili yükleniyor: chris-wood
  Veri çekildi: Chris Wood | Takım: Nottingham Forest | Maç: 36 | Gol: 20
  (5) Oyuncu profili yükleniyor: bryan-mbeumo
  Veri çekildi: Bryan Mbeumo | Takım: Brentford | Maç: 38 | Gol: 20
  (6) Oyuncu profili yükleniyor: yoane-wissa
  Veri çekildi: Yoane Wissa | Takım: Brentford | Maç: 35 | Gol: 19
  (7) Oyu

## Asistler

In [21]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import re


STATS_URL = "https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/goal_assist/premier-league"
OUTPUT_CSV_FILE = "player_stats.csv"
PLAYERS_PER_PAGE = 10
PAGES_TO_SCRAPE = 3


options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument("log-level=3")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

print("Selenium WebDriver başlatıldı.")

all_players_data = []

def safe_int_extract(text):
    """Metinden sayısal değerleri güvenli bir şekilde çıkarır"""
    if not text:
        return 0
    
    
    numbers = re.findall(r'\d+', text.strip())
    if numbers:
        return int(numbers[0])
    return 0

def get_stat_value_by_label(driver, wait, stat_label):
    """Etiket adıyla istatistik değerini alır (css-170fd60-StatValue class'ından)"""
    try:
        
        label_xpath = f"//span[contains(@class, 'css-1m9s03m-StatTitle') and text()='{stat_label}']/parent::div/preceding-sibling::div[contains(@class, 'css-170fd60-StatValue')]//span"
        element = wait.until(EC.visibility_of_element_located((By.XPATH, label_xpath)))
        value_text = element.text.strip()
        return safe_int_extract(value_text)
    
    except (TimeoutException, NoSuchElementException):
        
        try:
            
            stat_box_xpath = f"//div[contains(@class, 'css-8ko4sn-StatBox')]//span[contains(@class, 'css-1m9s03m-StatTitle') and text()='{stat_label}']/parent::div/preceding-sibling::div[contains(@class, 'css-170fd60-StatValue')]//span"
            element = driver.find_element(By.XPATH, stat_box_xpath)
            value_text = element.text.strip()
            return safe_int_extract(value_text)
        except:
            pass
        
        
        try:
            stat_boxes = driver.find_elements(By.CSS_SELECTOR, "div.css-8ko4sn-StatBox")
            
            for box in stat_boxes:
                try:
                    
                    title_element = box.find_element(By.CSS_SELECTOR, "span.css-1m9s03m-StatTitle")
                    if title_element.text.strip() == stat_label:
                        
                        value_element = box.find_element(By.CSS_SELECTOR, "div.css-170fd60-StatValue span")
                        return safe_int_extract(value_element.text)
                except:
                    continue
                    
        except Exception as e:
            print(f"    Son alternatif yöntem hatası: {e}")
        
        print(f"    Uyarı: {stat_label} değeri bulunamadı, 0 olarak ayarlandı")
        return 0

try:
    driver.get(STATS_URL)
    print(f"Hedef URL'ye gidiliyor: {STATS_URL}")
    
    wait = WebDriverWait(driver, 15)

    try:
        print("Çerez onay penceresi bekleniyor...")
        accept_cookies_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_cookies_button.click()
        print("Çerezler kabul edildi.")
        time.sleep(2)
    except TimeoutException:
        print("Çerez onay penceresi bulunamadı, devam ediliyor.")

    
    for page_num in range(PAGES_TO_SCRAPE):
        current_page = page_num + 1
        print(f"\n--- Sayfa {current_page} işleniyor ---")

        
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/tr/players/']")))
        
        player_rows = driver.find_elements(By.CSS_SELECTOR, "a[href*='/tr/players/']")
        player_urls_on_page = [row.get_attribute('href') for row in player_rows[:PLAYERS_PER_PAGE]]
        
        print(f"  Bu sayfada {len(player_urls_on_page)} oyuncu bulundu")
        
        for i, player_url in enumerate(player_urls_on_page):
            player_data = {}
            
            try:
                driver.get(player_url)
                
                rank = page_num * PLAYERS_PER_PAGE + i + 1
                print(f"  ({rank}) Oyuncu profili yükleniyor: {player_url.split('/')[-1]}")

               
                player_name_element = wait.until(EC.visibility_of_element_located((By.TAG_NAME, "h1")))
                player_data['Player'] = player_name_element.text.strip()

                
                try:
                    team_selectors = [
                        "div.css-14k6s2u-TeamCSS",
                        "div[class*='TeamCSS']",
                        "div[class*='team']"
                    ]
                    
                    team_name = None
                    for selector in team_selectors:
                        try:
                            team_element = driver.find_element(By.CSS_SELECTOR, selector)
                            team_name = team_element.text.strip()
                            if team_name:
                                break
                        except:
                            continue
                    
                    player_data['Team'] = team_name if team_name else "Bilinmeyen"
                        
                except Exception as e:
                    player_data['Team'] = "Bilinmeyen"
                    print(f"    Uyarı: Takım adı alınırken hata: {e}")
                
                
                time.sleep(2)
                
                
                player_data['Assists'] = get_stat_value_by_label(driver, wait, 'Asistler')
                player_data['Matches'] = get_stat_value_by_label(driver, wait, 'Maçlar')

                print(f"  Veri çekildi: {player_data['Player']} | Takım: {player_data['Team']} | Maç: {player_data['Matches']} | Asist: {player_data['Assists']}")
                all_players_data.append(player_data)

            except Exception as e:
                print(f"  Hata: {player_url.split('/')[-1]} için veri çekilemedi. Hata: {e}")
                
                if 'Player' in player_data:
                    player_data.setdefault('Team', 'Bilinmeyen')
                    player_data.setdefault('Assists', 0)
                    player_data.setdefault('Matches', 0)
                    all_players_data.append(player_data)
            
           
            try:
                driver.back()
                
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/tr/players/']")))
                time.sleep(1)
            except Exception as e:
                print(f"  Geri dönüş hatası: {e}")
                
                driver.get(STATS_URL)
                wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/tr/players/']")))
                time.sleep(2)

        
        if page_num < PAGES_TO_SCRAPE - 1:
            print(f"\n{current_page}. sayfadan {current_page + 1}. sayfaya geçiliyor...")
            
            try:
               
                next_button_selectors = [
                    "//button[contains(text(), 'Sonraki')]",
                    "//button[contains(., 'Sonraki')]",
                    "//button[contains(@class, 'next')]",
                    "//button[contains(@aria-label, 'next')]",
                    "//button[contains(@aria-label, 'Sonraki')]"
                ]
                
                next_button = None
                for selector in next_button_selectors:
                    try:
                        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                        print(f"  'Sonraki' butonu bulundu: {selector}")
                        break
                    except TimeoutException:
                        continue
                
                if next_button:
                    
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    time.sleep(1)
                    
                    
                    try:
                        next_button.click()
                    except:
                        
                        driver.execute_script("arguments[0].click();", next_button)
                    
                    print(f"  Başarıyla {current_page + 1}. sayfaya geçildi.")
                    time.sleep(3)  
                    
                   
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/tr/players/']")))
                    
                else:
                    print("  'Sonraki' butonu bulunamadı. İşlem sonlandırılıyor.")
                    break

            except Exception as e:
                print(f"  Sonraki sayfaya geçerken hata: {e}")
                print("  İşlem sonlandırılıyor.")
                break

finally:
    driver.quit()
    print("\nSelenium WebDriver kapatıldı.")


if all_players_data:
    df = pd.DataFrame(all_players_data)
    
    
    df.insert(0, 'Rank', range(1, 1 + len(df)))
    
    
    df = df[['Rank', 'Player', 'Team', 'Matches', 'Assists']]
    
   
    df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')
    print(f"\nVeriler başarıyla '{OUTPUT_CSV_FILE}' dosyasına kaydedildi.")
    print(f"Toplam {len(df)} oyuncu verisi toplandı.")
    print("\n--- İlk 5 Satır ---")
    print(df.head())
    print("\n--- Son 5 Satır ---")
    print(df.tail())
else:
    print("\nHiçbir veri toplanamadığı için CSV dosyası oluşturulmadı.")

Selenium WebDriver başlatıldı.
Hedef URL'ye gidiliyor: https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/goal_assist/premier-league
Çerez onay penceresi bekleniyor...
Çerez onay penceresi bulunamadı, devam ediliyor.

--- Sayfa 1 işleniyor ---
  Bu sayfada 10 oyuncu bulundu
  (1) Oyuncu profili yükleniyor: mohamed-salah
  Veri çekildi: Mohamed Salah | Takım: Liverpool | Maç: 38 | Asist: 18
  (2) Oyuncu profili yükleniyor: jacob-murphy
  Veri çekildi: Jacob Murphy | Takım: Newcastle United | Maç: 35 | Asist: 12
  (3) Oyuncu profili yükleniyor: anthony-elanga
  Veri çekildi: Anthony Elanga | Takım: Nottingham Forest | Maç: 38 | Asist: 11
  (4) Oyuncu profili yükleniyor: bruno-fernandes
  Veri çekildi: Bruno Fernandes | Takım: Manchester United | Maç: 36 | Asist: 10
  (5) Oyuncu profili yükleniyor: bukayo-saka
  Veri çekildi: Bukayo Saka | Takım: Arsenal | Maç: 25 | Asist: 10
  (6) Oyuncu profili yükleniyor: mikkel-damsgaard
  Veri çekildi: Mikkel Damsgaard | Takım: Brentford

## 90 Dk Başına Gol

In [28]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import re

STATS_URL = "https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/goals_per_90/premier-league"
OUTPUT_CSV_FILE = "goals_per_90_stats.csv"
PAGES_TO_SCRAPE = 3


options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument("log-level=3")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

print("Selenium WebDriver başlatıldı.")

all_players_data = []

def safe_float_extract(text):
    """Metinden ondalık sayısal değerleri güvenli bir şekilde çıkarır"""
    if not text:
        return 0.0
    
    
    text = text.replace(',', '.')
    numbers = re.findall(r'\d+\.?\d*', text.strip())
    if numbers:
        return float(numbers[0])
    return 0.0

try:
    driver.get(STATS_URL)
    print(f"Hedef URL'ye gidiliyor: {STATS_URL}")
    
    wait = WebDriverWait(driver, 15)

    try:
        print("Çerez onay penceresi bekleniyor...")
        accept_cookies_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_cookies_button.click()
        print("Çerezler kabul edildi.")
        time.sleep(2)
    except TimeoutException:
        print("Çerez onay penceresi bulunamadı, devam ediliyor.")

    
    for page_num in range(PAGES_TO_SCRAPE):
        current_page = page_num + 1
        print(f"\n--- Sayfa {current_page} işleniyor ---")

        
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")))
        
        
        player_rows = driver.find_elements(By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")
        
        print(f"  Bu sayfada {len(player_rows)} oyuncu bulundu")
        
        for i, row in enumerate(player_rows):
            try:
                
                player_name_element = row.find_element(By.CSS_SELECTOR, "span.css-cfc8cy-TeamOrPlayerName")
                player_name = player_name_element.text.strip()
                
                
                stat_value_element = row.find_element(By.CSS_SELECTOR, "span.css-wo228r-StatValue span")
                stat_value = safe_float_extract(stat_value_element.text)
                
                
                rank_element = row.find_element(By.CSS_SELECTOR, "span.css-twq2db-Rank")
                rank = int(rank_element.text.strip())
                
                player_data = {
                    'Rank': rank,
                    'Player': player_name,
                    'Goals_Per_90': stat_value
                }
                
                print(f"  ({rank}) {player_name} | 90 dk başına gol: {stat_value}")
                all_players_data.append(player_data)
                
            except Exception as e:
                print(f"  Hata: {i+1}. oyuncu için veri çekilemedi. Hata: {e}")
                continue

        
        if page_num < PAGES_TO_SCRAPE - 1:
            print(f"\n{current_page}. sayfadan {current_page + 1}. sayfaya geçiliyor...")
            
            try:
                
                next_button_selectors = [
                    "//button[contains(text(), 'Sonraki')]",
                    "//button[contains(., 'Sonraki')]",
                    "//button[contains(@class, 'next')]",
                    "//button[contains(@aria-label, 'next')]",
                    "//button[contains(@aria-label, 'Sonraki')]"
                ]
                
                next_button = None
                for selector in next_button_selectors:
                    try:
                        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                        print(f"  'Sonraki' butonu bulundu: {selector}")
                        break
                    except TimeoutException:
                        continue
                
                if next_button:
                    
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    time.sleep(1)
                    
                    
                    try:
                        next_button.click()
                    except:
                        
                        driver.execute_script("arguments[0].click();", next_button)
                    
                    print(f"  Başarıyla {current_page + 1}. sayfaya geçildi.")
                    time.sleep(3)  
                    
                    
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")))
                    
                else:
                    print("  'Sonraki' butonu bulunamadı. İşlem sonlandırılıyor.")
                    break

            except Exception as e:
                print(f"  Sonraki sayfaya geçerken hata: {e}")
                print("  İşlem sonlandırılıyor.")
                break

finally:
    driver.quit()
    print("\nSelenium WebDriver kapatıldı.")


if all_players_data:
    df = pd.DataFrame(all_players_data)
    
   
    df = df[['Rank', 'Player', 'Goals_Per_90']]
    
    
    df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')
    print(f"\nVeriler başarıyla '{OUTPUT_CSV_FILE}' dosyasına kaydedildi.")
    print(f"Toplam {len(df)} oyuncu verisi toplandı.")
    print("\n--- İlk 5 Satır ---")
    print(df.head())
    print("\n--- Son 5 Satır ---")
    print(df.tail())
else:
    print("\nHiçbir veri toplanamadığı için CSV dosyası oluşturulmadı.")

Selenium WebDriver başlatıldı.
Hedef URL'ye gidiliyor: https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/goals_per_90/premier-league
Çerez onay penceresi bekleniyor...
Çerez onay penceresi bulunamadı, devam ediliyor.

--- Sayfa 1 işleniyor ---
  Bu sayfada 9 oyuncu bulundu
  (1) Jhon Duran | 90 dk başına gol: 1.01
  (2) Mohamed Salah | 90 dk başına gol: 0.77
  (3) Rodrigo Muniz | 90 dk başına gol: 0.76
  (4) Alexander Isak | 90 dk başına gol: 0.75
  (5) Erling Haaland | 90 dk başına gol: 0.72
  (6) Chris Wood | 90 dk başına gol: 0.6
  (7) Yoane Wissa | 90 dk başına gol: 0.58
  (8) Ollie Watkins | 90 dk başına gol: 0.55
  (9) Bryan Mbeumo | 90 dk başına gol: 0.53

1. sayfadan 2. sayfaya geçiliyor...
  'Sonraki' butonu bulundu: //button[contains(., 'Sonraki')]
  Başarıyla 2. sayfaya geçildi.

--- Sayfa 2 işleniyor ---
  Bu sayfada 9 oyuncu bulundu
  (11) Luis Diaz | 90 dk başına gol: 0.49
  (12) Jørgen Strand Larsen | 90 dk başına gol: 0.48
  (13) Jean-Philippe Mateta | 90 

## XG ( Expected Goals)

In [31]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import re


STATS_URL = "https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/expected_goals/premier-league"
OUTPUT_CSV_FILE = "expected_goals_stats.csv"
PAGES_TO_SCRAPE = 3


options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument("log-level=3")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

print("Selenium WebDriver başlatıldı.")

all_players_data = []

def safe_float_extract(text):
    """Metinden ondalık sayısal değerleri güvenli bir şekilde çıkarır"""
    if not text:
        return 0.0
    
 
    text = text.replace(',', '.')
    numbers = re.findall(r'\d+\.?\d*', text.strip())
    if numbers:
        return float(numbers[0])
    return 0.0

try:
    driver.get(STATS_URL)
    print(f"Hedef URL'ye gidiliyor: {STATS_URL}")
    
    wait = WebDriverWait(driver, 15)

    try:
        print("Çerez onay penceresi bekleniyor...")
        accept_cookies_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_cookies_button.click()
        print("Çerezler kabul edildi.")
        time.sleep(2)
    except TimeoutException:
        print("Çerez onay penceresi bulunamadı, devam ediliyor.")

   
    for page_num in range(PAGES_TO_SCRAPE):
        current_page = page_num + 1
        print(f"\n--- Sayfa {current_page} işleniyor ---")

       
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")))
        
        
        player_rows = driver.find_elements(By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")
        
        print(f"  Bu sayfada {len(player_rows)} oyuncu bulundu")
        
        for i, row in enumerate(player_rows):
            try:
                
                player_name_element = row.find_element(By.CSS_SELECTOR, "span.css-cfc8cy-TeamOrPlayerName")
                player_name = player_name_element.text.strip()
                
                
                stat_value_element = row.find_element(By.CSS_SELECTOR, "span.css-wo228r-StatValue span")
                stat_value = safe_float_extract(stat_value_element.text)
                
                
                rank_element = row.find_element(By.CSS_SELECTOR, "span.css-twq2db-Rank")
                rank = int(rank_element.text.strip())
                
                player_data = {
                    'Rank': rank,
                    'Player': player_name,
                    'Expected_Goals': stat_value
                }
                
                print(f"  ({rank}) {player_name} | Expected Goals: {stat_value}")
                all_players_data.append(player_data)
                
            except Exception as e:
                print(f"  Hata: {i+1}. oyuncu için veri çekilemedi. Hata: {e}")
                continue

        
        if page_num < PAGES_TO_SCRAPE - 1:
            print(f"\n{current_page}. sayfadan {current_page + 1}. sayfaya geçiliyor...")
            
            try:
                
                next_button_selectors = [
                    "//button[contains(text(), 'Sonraki')]",
                    "//button[contains(., 'Sonraki')]",
                    "//button[contains(@class, 'next')]",
                    "//button[contains(@aria-label, 'next')]",
                    "//button[contains(@aria-label, 'Sonraki')]"
                ]
                
                next_button = None
                for selector in next_button_selectors:
                    try:
                        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                        print(f"  'Sonraki' butonu bulundu: {selector}")
                        break
                    except TimeoutException:
                        continue
                
                if next_button:
                    
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    time.sleep(1)
                    
                   
                    try:
                        next_button.click()
                    except:
                        
                        driver.execute_script("arguments[0].click();", next_button)
                    
                    print(f"  Başarıyla {current_page + 1}. sayfaya geçildi.")
                    time.sleep(3)  
                    
                    
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")))
                    
                else:
                    print("  'Sonraki' butonu bulunamadı. İşlem sonlandırılıyor.")
                    break

            except Exception as e:
                print(f"  Sonraki sayfaya geçerken hata: {e}")
                print("  İşlem sonlandırılıyor.")
                break

finally:
    driver.quit()
    print("\nSelenium WebDriver kapatıldı.")


if all_players_data:
    df = pd.DataFrame(all_players_data)
    
    
    df = df[['Rank', 'Player', 'Expected_Goals']]
    
    
    df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')
    print(f"\nVeriler başarıyla '{OUTPUT_CSV_FILE}' dosyasına kaydedildi.")
    print(f"Toplam {len(df)} oyuncu verisi toplandı.")
    print("\n--- İlk 5 Satır ---")
    print(df.head())
    print("\n--- Son 5 Satır ---")
    print(df.tail())
else:
    print("\nHiçbir veri toplanamadığı için CSV dosyası oluşturulmadı.")

Selenium WebDriver başlatıldı.
Hedef URL'ye gidiliyor: https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/expected_goals/premier-league
Çerez onay penceresi bekleniyor...
Çerez onay penceresi bulunamadı, devam ediliyor.

--- Sayfa 1 işleniyor ---
  Bu sayfada 9 oyuncu bulundu
  (1) Mohamed Salah | Expected Goals: 25.4
  (2) Erling Haaland | Expected Goals: 22.0
  (3) Alexander Isak | Expected Goals: 20.4
  (4) Yoane Wissa | Expected Goals: 18.6
  (5) Cole Palmer | Expected Goals: 17.3
  (6) Ollie Watkins | Expected Goals: 15.5
  (7) Jean-Philippe Mateta | Expected Goals: 13.5
  (8) Chris Wood | Expected Goals: 13.4
  (9) Evanilson | Expected Goals: 12.5

1. sayfadan 2. sayfaya geçiliyor...
  'Sonraki' butonu bulundu: //button[contains(., 'Sonraki')]
  Başarıyla 2. sayfaya geçildi.

--- Sayfa 2 işleniyor ---
  Bu sayfada 9 oyuncu bulundu
  (11) Nicolas Jackson | Expected Goals: 12.3
  (12) Luis Diaz | Expected Goals: 12.0
  (13) Raul Jiménez | Expected Goals: 12.0
  (14) Ja

## 90 Dakika Başına Şut

In [35]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import re


STATS_URL = "https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/total_scoring_att/premier-league"
OUTPUT_CSV_FILE = "total_scoring_attempts_stats.csv"
PAGES_TO_SCRAPE = 3

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument("log-level=3")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

print("Selenium WebDriver başlatıldı.")

all_players_data = []

def safe_float_extract(text):
    """Metinden ondalık sayısal değerleri güvenli bir şekilde çıkarır"""
    if not text:
        return 0.0
    
    
    text = text.replace(',', '.')
    numbers = re.findall(r'\d+\.?\d*', text.strip())
    if numbers:
        return float(numbers[0])
    return 0.0

try:
    driver.get(STATS_URL)
    print(f"Hedef URL'ye gidiliyor: {STATS_URL}")
    
    wait = WebDriverWait(driver, 15)

    try:
        print("Çerez onay penceresi bekleniyor...")
        accept_cookies_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_cookies_button.click()
        print("Çerezler kabul edildi.")
        time.sleep(2)
    except TimeoutException:
        print("Çerez onay penceresi bulunamadı, devam ediliyor.")

   
    for page_num in range(PAGES_TO_SCRAPE):
        current_page = page_num + 1
        print(f"\n--- Sayfa {current_page} işleniyor ---")

       
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")))
        
        
        player_rows = driver.find_elements(By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")
        
        print(f"  Bu sayfada {len(player_rows)} oyuncu bulundu")
        
        for i, row in enumerate(player_rows):
            try:
                
                player_name_element = row.find_element(By.CSS_SELECTOR, "span.css-cfc8cy-TeamOrPlayerName")
                player_name = player_name_element.text.strip()
                
               
                stat_value_element = row.find_element(By.CSS_SELECTOR, "span.css-wo228r-StatValue span")
                stat_value = safe_float_extract(stat_value_element.text)
                
               
                rank_element = row.find_element(By.CSS_SELECTOR, "span.css-twq2db-Rank")
                rank = int(rank_element.text.strip())
                
                player_data = {
                    'Rank': rank,
                    'Player': player_name,
                    'Total_Scoring_Attempts': stat_value
                }
                
                print(f"  ({rank}) {player_name} | Total Scoring Attempts: {stat_value}")
                all_players_data.append(player_data)
                
            except Exception as e:
                print(f"  Hata: {i+1}. oyuncu için veri çekilemedi. Hata: {e}")
                continue

       
        if page_num < PAGES_TO_SCRAPE - 1:
            print(f"\n{current_page}. sayfadan {current_page + 1}. sayfaya geçiliyor...")
            
            try:
               
                next_button_selectors = [
                    "//button[contains(text(), 'Sonraki')]",
                    "//button[contains(., 'Sonraki')]",
                    "//button[contains(@class, 'next')]",
                    "//button[contains(@aria-label, 'next')]",
                    "//button[contains(@aria-label, 'Sonraki')]"
                ]
                
                next_button = None
                for selector in next_button_selectors:
                    try:
                        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                        print(f"  'Sonraki' butonu bulundu: {selector}")
                        break
                    except TimeoutException:
                        continue
                
                if next_button:
                   
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    time.sleep(1)
                    
                    
                    try:
                        next_button.click()
                    except:
                        
                        driver.execute_script("arguments[0].click();", next_button)
                    
                    print(f"  Başarıyla {current_page + 1}. sayfaya geçildi.")
                    time.sleep(3)  
                    
                    
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")))
                    
                else:
                    print("  'Sonraki' butonu bulunamadı. İşlem sonlandırılıyor.")
                    break

            except Exception as e:
                print(f"  Sonraki sayfaya geçerken hata: {e}")
                print("  İşlem sonlandırılıyor.")
                break

finally:
    driver.quit()
    print("\nSelenium WebDriver kapatıldı.")


if all_players_data:
    df = pd.DataFrame(all_players_data)
    
    
    df = df[['Rank', 'Player', 'Total_Scoring_Attempts']]
    
    
    df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')
    print(f"\nVeriler başarıyla '{OUTPUT_CSV_FILE}' dosyasına kaydedildi.")
    print(f"Toplam {len(df)} oyuncu verisi toplandı.")
    print("\n--- İlk 5 Satır ---")
    print(df.head())
    print("\n--- Son 5 Satır ---")
    print(df.tail())
else:
    print("\nHiçbir veri toplanamadığı için CSV dosyası oluşturulmadı.")

Selenium WebDriver başlatıldı.
Hedef URL'ye gidiliyor: https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/total_scoring_att/premier-league
Çerez onay penceresi bekleniyor...
Çerez onay penceresi bulunamadı, devam ediliyor.

--- Sayfa 1 işleniyor ---
  Bu sayfada 9 oyuncu bulundu
  (1) Jhon Duran | Total Scoring Attempts: 4.5
  (2) Julio Enciso | Total Scoring Attempts: 4.0
  (3) Matheus Cunha | Total Scoring Attempts: 3.8
  (4) Diogo Jota | Total Scoring Attempts: 3.8
  (5) Edward Nketiah | Total Scoring Attempts: 3.8
  (6) Rodrigo Muniz | Total Scoring Attempts: 3.7
  (7) Mohamed Salah | Total Scoring Attempts: 3.5
  (8) Erling Haaland | Total Scoring Attempts: 3.5
  (9) Cole Palmer | Total Scoring Attempts: 3.5

1. sayfadan 2. sayfaya geçiliyor...
  'Sonraki' butonu bulundu: //button[contains(., 'Sonraki')]
  Başarıyla 2. sayfaya geçildi.

--- Sayfa 2 işleniyor ---
  Bu sayfada 9 oyuncu bulundu
  (11) Bukayo Saka | Total Scoring Attempts: 3.5
  (12) Antoine Semenyo | Tot

## 90 Dakika Başına İsabetli Pas

In [38]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import re


STATS_URL = "https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/accurate_pass/premier-league"
OUTPUT_CSV_FILE = "accurate_pass_stats.csv"
PAGES_TO_SCRAPE = 3


options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument("log-level=3")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

print("Selenium WebDriver başlatıldı.")

all_players_data = []

def safe_float_extract(text):
    """Metinden ondalık sayısal değerleri güvenli bir şekilde çıkarır"""
    if not text:
        return 0.0
    
    text = text.replace(',', '.')
    numbers = re.findall(r'\d+\.?\d*', text.strip())
    if numbers:
        return float(numbers[0])
    return 0.0

try:
    driver.get(STATS_URL)
    print(f"Hedef URL'ye gidiliyor: {STATS_URL}")
    
    wait = WebDriverWait(driver, 15)

    try:
        print("Çerez onay penceresi bekleniyor...")
        accept_cookies_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_cookies_button.click()
        print("Çerezler kabul edildi.")
        time.sleep(2)
    except TimeoutException:
        print("Çerez onay penceresi bulunamadı, devam ediliyor.")

   
    for page_num in range(PAGES_TO_SCRAPE):
        current_page = page_num + 1
        print(f"\n--- Sayfa {current_page} işleniyor ---")

        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")))
        
        player_rows = driver.find_elements(By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")
        
        print(f"  Bu sayfada {len(player_rows)} oyuncu bulundu")
        
        for i, row in enumerate(player_rows):
            try:
                player_name_element = row.find_element(By.CSS_SELECTOR, "span.css-cfc8cy-TeamOrPlayerName")
                player_name = player_name_element.text.strip()
                
                stat_value_element = row.find_element(By.CSS_SELECTOR, "span.css-wo228r-StatValue span")
                stat_value = safe_float_extract(stat_value_element.text)
                
                rank_element = row.find_element(By.CSS_SELECTOR, "span.css-twq2db-Rank")
                rank = int(rank_element.text.strip())
                
               
                player_data = {
                    'Rank': rank,
                    'Player': player_name,
                    'Accurate_Passes': stat_value
                }
                
                
                print(f"  ({rank}) {player_name} | Accurate Passes: {stat_value}")
                all_players_data.append(player_data)
                
            except Exception as e:
                print(f"  Hata: {i+1}. oyuncu için veri çekilemedi. Hata: {e}")
                continue

        if page_num < PAGES_TO_SCRAPE - 1:
            print(f"\n{current_page}. sayfadan {current_page + 1}. sayfaya geçiliyor...")
            
            try:
                next_button_selectors = [
                    "//button[contains(text(), 'Sonraki')]",
                    "//button[contains(., 'Sonraki')]",
                    "//button[contains(@class, 'next')]",
                    "//button[contains(@aria-label, 'next')]",
                    "//button[contains(@aria-label, 'Sonraki')]"
                ]
                
                next_button = None
                for selector in next_button_selectors:
                    try:
                        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                        print(f"  'Sonraki' butonu bulundu: {selector}")
                        break
                    except TimeoutException:
                        continue
                
                if next_button:
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    time.sleep(1)
                    
                    try:
                        next_button.click()
                    except:
                        driver.execute_script("arguments[0].click();", next_button)
                    
                    print(f"  Başarıyla {current_page + 1}. sayfaya geçildi.")
                    time.sleep(3)
                    
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")))
                    
                else:
                    print("  'Sonraki' butonu bulunamadı. İşlem sonlandırılıyor.")
                    break

            except Exception as e:
                print(f"  Sonraki sayfaya geçerken hata: {e}")
                print("  İşlem sonlandırılıyor.")
                break

finally:
    driver.quit()
    print("\nSelenium WebDriver kapatıldı.")

if all_players_data:
    df = pd.DataFrame(all_players_data)
    
   
    df = df[['Rank', 'Player', 'Accurate_Passes']]
    
    df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')
    print(f"\nVeriler başarıyla '{OUTPUT_CSV_FILE}' dosyasına kaydedildi.")
    print(f"Toplam {len(df)} oyuncu verisi toplandı.")
    print("\n--- İlk 5 Satır ---")
    print(df.head())
    print("\n--- Son 5 Satır ---")
    print(df.tail())
else:
    print("\nHiçbir veri toplanamadığı için CSV dosyası oluşturulmadı.")

Selenium WebDriver başlatıldı.
Hedef URL'ye gidiliyor: https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/accurate_pass/premier-league
Çerez onay penceresi bekleniyor...
Çerez onay penceresi bulunamadı, devam ediliyor.

--- Sayfa 1 işleniyor ---
  Bu sayfada 9 oyuncu bulundu
  (1) Ruben Dias | Accurate Passes: 85.9
  (2) Tom Cairney | Accurate Passes: 82.2
  (3) Lewis Dunk | Accurate Passes: 76.4
  (4) Manuel Akanji | Accurate Passes: 72.4
  (5) Virgil van Dijk | Accurate Passes: 72.4
  (6) William Saliba | Accurate Passes: 71.3
  (7) Mateo Kovacic | Accurate Passes: 71.2
  (8) Tosin Adarabioyo | Accurate Passes: 68.9
  (9) Levi Colwill | Accurate Passes: 66.5

1. sayfadan 2. sayfaya geçiliyor...
  'Sonraki' butonu bulundu: //button[contains(., 'Sonraki')]
  Başarıyla 2. sayfaya geçildi.

--- Sayfa 2 işleniyor ---
  Bu sayfada 9 oyuncu bulundu
  (11) Wataru Endo | Accurate Passes: 61.4
  (12) Ilkay Gündogan | Accurate Passes: 61.0
  (13) Jan Paul van Hecke | Accurate Passe

## Yaratılan Gol Fırsatı

In [41]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import re


STATS_URL = "https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/big_chance_created/premier-league"
OUTPUT_CSV_FILE = "big_chances_created_stats.csv"
PAGES_TO_SCRAPE = 3


options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument("log-level=3")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

print("Selenium WebDriver başlatıldı.")

all_players_data = []

def safe_float_extract(text):
    """Metinden ondalık sayısal değerleri güvenli bir şekilde çıkarır"""
    if not text:
        return 0.0
    
    text = text.replace(',', '.')
    numbers = re.findall(r'\d+\.?\d*', text.strip())
    if numbers:
        return float(numbers[0])
    return 0.0

try:
    driver.get(STATS_URL)
    print(f"Hedef URL'ye gidiliyor: {STATS_URL}")
    
    wait = WebDriverWait(driver, 15)

    try:
        print("Çerez onay penceresi bekleniyor...")
        accept_cookies_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_cookies_button.click()
        print("Çerezler kabul edildi.")
        time.sleep(2)
    except TimeoutException:
        print("Çerez onay penceresi bulunamadı, devam ediliyor.")

    
    for page_num in range(PAGES_TO_SCRAPE):
        current_page = page_num + 1
        print(f"\n--- Sayfa {current_page} işleniyor ---")

        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")))
        
        player_rows = driver.find_elements(By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")
        
        print(f"  Bu sayfada {len(player_rows)} oyuncu bulundu")
        
        for i, row in enumerate(player_rows):
            try:
                player_name_element = row.find_element(By.CSS_SELECTOR, "span.css-cfc8cy-TeamOrPlayerName")
                player_name = player_name_element.text.strip()
                
                stat_value_element = row.find_element(By.CSS_SELECTOR, "span.css-wo228r-StatValue span")
                stat_value = safe_float_extract(stat_value_element.text)
                
                rank_element = row.find_element(By.CSS_SELECTOR, "span.css-twq2db-Rank")
                rank = int(rank_element.text.strip())
                
                
                player_data = {
                    'Rank': rank,
                    'Player': player_name,
                    'Big_Chances_Created': stat_value
                }
                
                
                print(f"  ({rank}) {player_name} | Big Chances Created: {stat_value}")
                all_players_data.append(player_data)
                
            except Exception as e:
                print(f"  Hata: {i+1}. oyuncu için veri çekilemedi. Hata: {e}")
                continue

        if page_num < PAGES_TO_SCRAPE - 1:
            print(f"\n{current_page}. sayfadan {current_page + 1}. sayfaya geçiliyor...")
            
            try:
                next_button_selectors = [
                    "//button[contains(text(), 'Sonraki')]",
                    "//button[contains(., 'Sonraki')]",
                    "//button[contains(@class, 'next')]",
                    "//button[contains(@aria-label, 'next')]",
                    "//button[contains(@aria-label, 'Sonraki')]"
                ]
                
                next_button = None
                for selector in next_button_selectors:
                    try:
                        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                        print(f"  'Sonraki' butonu bulundu: {selector}")
                        break
                    except TimeoutException:
                        continue
                
                if next_button:
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    time.sleep(1)
                    
                    try:
                        next_button.click()
                    except:
                        driver.execute_script("arguments[0].click();", next_button)
                    
                    print(f"  Başarıyla {current_page + 1}. sayfaya geçildi.")
                    time.sleep(3)
                    
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")))
                    
                else:
                    print("  'Sonraki' butonu bulunamadı. İşlem sonlandırılıyor.")
                    break

            except Exception as e:
                print(f"  Sonraki sayfaya geçerken hata: {e}")
                print("  İşlem sonlandırılıyor.")
                break

finally:
    driver.quit()
    print("\nSelenium WebDriver kapatıldı.")

if all_players_data:
    df = pd.DataFrame(all_players_data)
    
    
    df = df[['Rank', 'Player', 'Big_Chances_Created']]
    
    df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')
    print(f"\nVeriler başarıyla '{OUTPUT_CSV_FILE}' dosyasına kaydedildi.")
    print(f"Toplam {len(df)} oyuncu verisi toplandı.")
    print("\n--- İlk 5 Satır ---")
    print(df.head())
    print("\n--- Son 5 Satır ---")
    print(df.tail())
else:
    print("\nHiçbir veri toplanamadığı için CSV dosyası oluşturulmadı.")

Selenium WebDriver başlatıldı.
Hedef URL'ye gidiliyor: https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/big_chance_created/premier-league
Çerez onay penceresi bekleniyor...
Çerez onay penceresi bulunamadı, devam ediliyor.

--- Sayfa 1 işleniyor ---
  Bu sayfada 9 oyuncu bulundu
  (1) Mohamed Salah | Big Chances Created: 27.0
  (2) Cole Palmer | Big Chances Created: 24.0
  (3) Bukayo Saka | Big Chances Created: 21.0
  (4) Bryan Mbeumo | Big Chances Created: 17.0
  (5) Ismaila Sarr | Big Chances Created: 17.0
  (6) Jacob Murphy | Big Chances Created: 16.0
  (7) Bruno Fernandes | Big Chances Created: 16.0
  (8) Morgan Rogers | Big Chances Created: 16.0
  (9) Heung-Min Son | Big Chances Created: 16.0

1. sayfadan 2. sayfaya geçiliyor...
  'Sonraki' butonu bulundu: //button[contains(., 'Sonraki')]
  Başarıyla 2. sayfaya geçildi.

--- Sayfa 2 işleniyor ---
  Bu sayfada 9 oyuncu bulundu
  (11) Youri Tielemans | Big Chances Created: 16.0
  (12) Mikkel Damsgaard | Big Chances Cre

## 90 Dakika Başına Başarılı Çalım

In [44]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import re


STATS_URL = "https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/won_contest/premier-league"
OUTPUT_CSV_FILE = "won_contests_stats.csv"
PAGES_TO_SCRAPE = 3


options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument("log-level=3")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

print("Selenium WebDriver başlatıldı.")

all_players_data = []

def safe_float_extract(text):
    """Metinden ondalık sayısal değerleri güvenli bir şekilde çıkarır"""
    if not text:
        return 0.0
    
    text = text.replace(',', '.')
    numbers = re.findall(r'\d+\.?\d*', text.strip())
    if numbers:
        return float(numbers[0])
    return 0.0

try:
    driver.get(STATS_URL)
    print(f"Hedef URL'ye gidiliyor: {STATS_URL}")
    
    wait = WebDriverWait(driver, 15)

    try:
        print("Çerez onay penceresi bekleniyor...")
        accept_cookies_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_cookies_button.click()
        print("Çerezler kabul edildi.")
        time.sleep(2)
    except TimeoutException:
        print("Çerez onay penceresi bulunamadı, devam ediliyor.")

    
    for page_num in range(PAGES_TO_SCRAPE):
        current_page = page_num + 1
        print(f"\n--- Sayfa {current_page} işleniyor ---")

        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")))
        
        player_rows = driver.find_elements(By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")
        
        print(f"  Bu sayfada {len(player_rows)} oyuncu bulundu")
        
        for i, row in enumerate(player_rows):
            try:
                player_name_element = row.find_element(By.CSS_SELECTOR, "span.css-cfc8cy-TeamOrPlayerName")
                player_name = player_name_element.text.strip()
                
                stat_value_element = row.find_element(By.CSS_SELECTOR, "span.css-wo228r-StatValue span")
                stat_value = safe_float_extract(stat_value_element.text)
                
                rank_element = row.find_element(By.CSS_SELECTOR, "span.css-twq2db-Rank")
                rank = int(rank_element.text.strip())
                
                
                player_data = {
                    'Rank': rank,
                    'Player': player_name,
                    'Won_Contests': stat_value
                }
                
                
                print(f"  ({rank}) {player_name} | Won Contests: {stat_value}")
                all_players_data.append(player_data)
                
            except Exception as e:
                print(f"  Hata: {i+1}. oyuncu için veri çekilemedi. Hata: {e}")
                continue

        if page_num < PAGES_TO_SCRAPE - 1:
            print(f"\n{current_page}. sayfadan {current_page + 1}. sayfaya geçiliyor...")
            
            try:
                next_button_selectors = [
                    "//button[contains(text(), 'Sonraki')]",
                    "//button[contains(., 'Sonraki')]",
                    "//button[contains(@class, 'next')]",
                    "//button[contains(@aria-label, 'next')]",
                    "//button[contains(@aria-label, 'Sonraki')]"
                ]
                
                next_button = None
                for selector in next_button_selectors:
                    try:
                        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                        print(f"  'Sonraki' butonu bulundu: {selector}")
                        break
                    except TimeoutException:
                        continue
                
                if next_button:
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    time.sleep(1)
                    
                    try:
                        next_button.click()
                    except:
                        driver.execute_script("arguments[0].click();", next_button)
                    
                    print(f"  Başarıyla {current_page + 1}. sayfaya geçildi.")
                    time.sleep(3)
                    
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")))
                    
                else:
                    print("  'Sonraki' butonu bulunamadı. İşlem sonlandırılıyor.")
                    break

            except Exception as e:
                print(f"  Sonraki sayfaya geçerken hata: {e}")
                print("  İşlem sonlandırılıyor.")
                break

finally:
    driver.quit()
    print("\nSelenium WebDriver kapatıldı.")

if all_players_data:
    df = pd.DataFrame(all_players_data)
    
   
    df = df[['Rank', 'Player', 'Won_Contests']]
    
    df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')
    print(f"\nVeriler başarıyla '{OUTPUT_CSV_FILE}' dosyasına kaydedildi.")
    print(f"Toplam {len(df)} oyuncu verisi toplandı.")
    print("\n--- İlk 5 Satır ---")
    print(df.head())
    print("\n--- Son 5 Satır ---")
    print(df.tail())
else:
    print("\nHiçbir veri toplanamadığı için CSV dosyası oluşturulmadı.")

Selenium WebDriver başlatıldı.
Hedef URL'ye gidiliyor: https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/won_contest/premier-league
Çerez onay penceresi bekleniyor...
Çerez onay penceresi bulunamadı, devam ediliyor.

--- Sayfa 1 işleniyor ---
  Bu sayfada 9 oyuncu bulundu
  (1) Jérémy Doku | Won Contests: 6.4
  (2) Ethan Nwaneri | Won Contests: 3.7
  (3) Mohammed Kudus | Won Contests: 3.2
  (4) Julio Enciso | Won Contests: 3.1
  (5) Ramon Sosa | Won Contests: 2.6
  (6) Savinho | Won Contests: 2.6
  (7) Iliman Ndiaye | Won Contests: 2.5
  (8) Kamaldeen Sulemana | Won Contests: 2.5
  (9) Adama Traoré | Won Contests: 2.4

1. sayfadan 2. sayfaya geçiliyor...
  'Sonraki' butonu bulundu: //button[contains(., 'Sonraki')]
  Başarıyla 2. sayfaya geçildi.

--- Sayfa 2 işleniyor ---
  Bu sayfada 9 oyuncu bulundu
  (11) Eberechi Eze | Won Contests: 2.3
  (12) Tyler Dibling | Won Contests: 2.2
  (13) Yankuba Minteh | Won Contests: 2.2
  (14) Bukayo Saka | Won Contests: 2.1
  (15) Math

## 90 Dakika Başına Başarılı İkili Mücadele

In [54]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import re


STATS_URL = "https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/won_tackle/premier-league"
OUTPUT_CSV_FILE = "won_tackles_stats.csv"
PAGES_TO_SCRAPE = 3


options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument("log-level=3")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

print("Selenium WebDriver başlatıldı.")

all_players_data = []

def safe_float_extract(text):
    """Metinden ondalık sayısal değerleri güvenli bir şekilde çıkarır"""
    if not text:
        return 0.0
    
    text = text.replace(',', '.')
    numbers = re.findall(r'\d+\.?\d*', text.strip())
    if numbers:
        return float(numbers[0])
    return 0.0

try:
    driver.get(STATS_URL)
    print(f"Hedef URL'ye gidiliyor: {STATS_URL}")
    
    wait = WebDriverWait(driver, 15)

    try:
        print("Çerez onay penceresi bekleniyor...")
        accept_cookies_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_cookies_button.click()
        print("Çerezler kabul edildi.")
        time.sleep(2)
    except TimeoutException:
        print("Çerez onay penceresi bulunamadı, devam ediliyor.")

   
    for page_num in range(PAGES_TO_SCRAPE):
        current_page = page_num + 1
        print(f"\n--- Sayfa {current_page} işleniyor ---")

        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")))
        
        player_rows = driver.find_elements(By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")
        
        print(f"  Bu sayfada {len(player_rows)} oyuncu bulundu")
        
        for i, row in enumerate(player_rows):
            try:
                player_name_element = row.find_element(By.CSS_SELECTOR, "span.css-cfc8cy-TeamOrPlayerName")
                player_name = player_name_element.text.strip()
                
                stat_value_element = row.find_element(By.CSS_SELECTOR, "span.css-wo228r-StatValue span")
                stat_value = safe_float_extract(stat_value_element.text)
                
                rank_element = row.find_element(By.CSS_SELECTOR, "span.css-twq2db-Rank")
                rank = int(rank_element.text.strip())
                
                
                player_data = {
                    'Rank': rank,
                    'Player': player_name,
                    'Won_Tackles': stat_value
                }
                
                
                print(f"  ({rank}) {player_name} | Won Tackles: {stat_value}")
                all_players_data.append(player_data)
                
            except Exception as e:
                print(f"  Hata: {i+1}. oyuncu için veri çekilemedi. Hata: {e}")
                continue

        if page_num < PAGES_TO_SCRAPE - 1:
            print(f"\n{current_page}. sayfadan {current_page + 1}. sayfaya geçiliyor...")
            
            try:
                next_button_selectors = [
                    "//button[contains(text(), 'Sonraki')]",
                    "//button[contains(., 'Sonraki')]",
                    "//button[contains(@class, 'next')]",
                    "//button[contains(@aria-label, 'next')]",
                    "//button[contains(@aria-label, 'Sonraki')]"
                ]
                
                next_button = None
                for selector in next_button_selectors:
                    try:
                        next_button = wait.until(EC.element_to_be_clickable((By.XPATH, selector)))
                        print(f"  'Sonraki' butonu bulundu: {selector}")
                        break
                    except TimeoutException:
                        continue
                
                if next_button:
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                    time.sleep(1)
                    
                    try:
                        next_button.click()
                    except:
                        driver.execute_script("arguments[0].click();", next_button)
                    
                    print(f"  Başarıyla {current_page + 1}. sayfaya geçildi.")
                    time.sleep(3)
                    
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")))
                    
                else:
                    print("  'Sonraki' butonu bulunamadı. İşlem sonlandırılıyor.")
                    break

            except Exception as e:
                print(f"  Sonraki sayfaya geçerken hata: {e}")
                print("  İşlem sonlandırılıyor.")
                break

finally:
    driver.quit()
    print("\nSelenium WebDriver kapatıldı.")

if all_players_data:
    df = pd.DataFrame(all_players_data)
    
   
    df = df[['Rank', 'Player', 'Won_Tackles']]
    
    df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')
    print(f"\nVeriler başarıyla '{OUTPUT_CSV_FILE}' dosyasına kaydedildi.")
    print(f"Toplam {len(df)} oyuncu verisi toplandı.")
    print("\n--- İlk 5 Satır ---")
    print(df.head())
    print("\n--- Son 5 Satır ---")
    print(df.tail())
else:
    print("\nHiçbir veri toplanamadığı için CSV dosyası oluşturulmadı.")

Selenium WebDriver başlatıldı.
Hedef URL'ye gidiliyor: https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/won_tackle/premier-league
Çerez onay penceresi bekleniyor...
Çerez onay penceresi bulunamadı, devam ediliyor.

--- Sayfa 1 işleniyor ---
  Bu sayfada 9 oyuncu bulundu
  (1) Casemiro | Won Tackles: 3.0
  (2) Mats Wieffer | Won Tackles: 3.0
  (3) Wataru Endo | Won Tackles: 2.8
  (4) Manuel Ugarte | Won Tackles: 2.5
  (5) Idrissa Gana Gueye | Won Tackles: 2.3
  (6) Tyler Adams | Won Tackles: 2.3
  (7) Justin Devenny | Won Tackles: 2.2
  (8) Daniel Munoz | Won Tackles: 2.2
  (9) Destiny Udogie | Won Tackles: 2.2

1. sayfadan 2. sayfaya geçiliyor...
  'Sonraki' butonu bulundu: //button[contains(., 'Sonraki')]
  Başarıyla 2. sayfaya geçildi.

--- Sayfa 2 işleniyor ---
  Bu sayfada 9 oyuncu bulundu
  (11) Noussair Mazraoui | Won Tackles: 2.1
  (12) Nicolás Dominguez | Won Tackles: 2.1
  (13) Moisés Caicedo | Won Tackles: 2.0
  (14) Neco Williams | Won Tackles: 2.0
  (15) Yves

## 90 Dakika Başına Top Kapma

In [56]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import re


STATS_URL = "https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/interception/premier-league"
OUTPUT_CSV_FILE = "interceptions_stats.csv"
PAGES_TO_SCRAPE = 3


options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument("log-level=3")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

print("Selenium WebDriver başlatıldı.")

all_players_data = []

def safe_float_extract(text):
    """Metinden ondalık sayısal değerleri güvenli bir şekilde çıkarır"""
    if not text:
        return 0.0
    
    text = text.replace(',', '.')
    numbers = re.findall(r'\d+\.?\d*', text.strip())
    if numbers:
        return float(numbers[0])
    return 0.0

try:
    driver.get(STATS_URL)
    print(f"Hedef URL'ye gidiliyor: {STATS_URL}")
    
    wait = WebDriverWait(driver, 15)

    try:
        print("Çerez onay penceresi bekleniyor...")
        accept_cookies_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_cookies_button.click()
        print("Çerezler kabul edildi.")
        time.sleep(2)
    except TimeoutException:
        print("Çerez onay penceresi bulunamadı, devam ediliyor.")

   
    for page_num in range(PAGES_TO_SCRAPE):
        current_page = page_num + 1
        print(f"\n--- Sayfa {current_page} işleniyor ---")

        try:
            
            print("  Oyuncu listesinin tam olarak yüklenmesi bekleniyor...")
            wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")) >= 10)
        except TimeoutException:
            
            print("  Sayfada 10'dan az oyuncu bulundu, mevcut olanlar işlenecek.")
            pass

        player_rows = driver.find_elements(By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")
        
        if not player_rows:
            print("  Bu sayfada hiç oyuncu bulunamadı. Döngü sonlandırılıyor.")
            break
            
        print(f"  Bu sayfada {len(player_rows)} oyuncu satırı bulundu.")

        
        print("  Listenin sonuna kaydırılarak tüm oyuncuların görünür olması sağlanıyor...")
        driver.execute_script("arguments[0].scrollIntoView();", player_rows[-1])
        time.sleep(1) 

        page_data_batch = []
        for i, row in enumerate(player_rows):
            try:
                player_name_element = row.find_element(By.CSS_SELECTOR, "span.css-cfc8cy-TeamOrPlayerName")
                player_name = player_name_element.text.strip()
                
                stat_value_element = row.find_element(By.CSS_SELECTOR, "span.css-wo228r-StatValue span")
                stat_value = safe_float_extract(stat_value_element.text)
                
                rank_element = row.find_element(By.CSS_SELECTOR, "span.css-twq2db-Rank")
                rank = int(rank_element.text.strip())
                
                player_data = {
                    'Rank': rank,
                    'Player': player_name,
                    'Interceptions': stat_value
                }
                page_data_batch.append(player_data)
                
            except Exception as e:
                print(f"  Hata: Bu sayfadaki {i+1}. oyuncu için veri çekilemedi. Hata: {e}")
                continue

        if page_data_batch:
            all_players_data.extend(page_data_batch)
            for p_data in page_data_batch:
                 print(f"  ({p_data['Rank']}) {p_data['Player']} | Interceptions: {p_data['Interceptions']}")
            print(f"  Sayfa {current_page} başarıyla işlendi ve {len(page_data_batch)} oyuncu eklendi.")

        if page_num < PAGES_TO_SCRAPE - 1:
            print(f"\n{current_page}. sayfadan {current_page + 1}. sayfaya geçiliyor...")
            
            try:
                next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Sonraki')]")))
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", next_button)
                print(f"  Başarıyla {current_page + 1}. sayfaya geçildi.")
                time.sleep(3)
                
            except Exception as e:
                print(f"  'Sonraki' butonu bulunamadı veya tıklanamadı. İşlem sonlandırılıyor. Hata: {e}")
                break

finally:
    driver.quit()
    print("\nSelenium WebDriver kapatıldı.")

if all_players_data:
    df = pd.DataFrame(all_players_data)
    df = df[['Rank', 'Player', 'Interceptions']]
    df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')
    print(f"\nVeriler başarıyla '{OUTPUT_CSV_FILE}' dosyasına kaydedildi.")
    print(f"Toplam {len(df)} oyuncu verisi toplandı.")
    print("\n--- İlk 5 Satır ---")
    print(df.head())
    print("\n--- Son 5 Satır ---")
    print(df.tail())
else:
    print("\nHiçbir veri toplanamadığı için CSV dosyası oluşturulmadı.")

Selenium WebDriver başlatıldı.
Hedef URL'ye gidiliyor: https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/interception/premier-league
Çerez onay penceresi bekleniyor...
Çerez onay penceresi bulunamadı, devam ediliyor.

--- Sayfa 1 işleniyor ---
  Oyuncu listesinin tam olarak yüklenmesi bekleniyor...
  Sayfada 10'dan az oyuncu bulundu, mevcut olanlar işlenecek.
  Bu sayfada 9 oyuncu satırı bulundu.
  Listenin sonuna kaydırılarak tüm oyuncuların görünür olması sağlanıyor...
  (1) Rodrigo Bentancur | Interceptions: 2.2
  (2) Wataru Endo | Interceptions: 2.1
  (3) Jan Bednarek | Interceptions: 2.0
  (4) Harry Maguire | Interceptions: 2.0
  (5) Aaron Wan-Bissaka | Interceptions: 1.9
  (6) Dean Huijsen | Interceptions: 1.9
  (7) Antonee Robinson | Interceptions: 1.8
  (8) Konstantinos Mavropanos | Interceptions: 1.8
  (9) Lucas Bergvall | Interceptions: 1.8
  Sayfa 1 başarıyla işlendi ve 9 oyuncu eklendi.

1. sayfadan 2. sayfaya geçiliyor...
  Başarıyla 2. sayfaya geçildi.

--- 

## Gol Yenmeyen Maç (Clean Sheet)

In [61]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import re


STATS_URL = "https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/clean_sheet/premier-league"
OUTPUT_CSV_FILE = "clean_sheets_stats.csv"
PAGES_TO_SCRAPE = 3


options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument("log-level=3")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

print("Selenium WebDriver başlatıldı.")

all_players_data = []

def safe_float_extract(text):
    """Metinden ondalık sayısal değerleri güvenli bir şekilde çıkarır"""
    if not text:
        return 0.0
    
    text = text.replace(',', '.')
    numbers = re.findall(r'\d+\.?\d*', text.strip())
    if numbers:
        return float(numbers[0])
    return 0.0

try:
    driver.get(STATS_URL)
    print(f"Hedef URL'ye gidiliyor: {STATS_URL}")
    
    wait = WebDriverWait(driver, 15)

    try:
        print("Çerez onay penceresi bekleniyor...")
        accept_cookies_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_cookies_button.click()
        print("Çerezler kabul edildi.")
        time.sleep(2)
    except TimeoutException:
        print("Çerez onay penceresi bulunamadı, devam ediliyor.")

 
    for page_num in range(PAGES_TO_SCRAPE):
        current_page = page_num + 1
        print(f"\n--- Sayfa {current_page} işleniyor ---")

        try:
            print("  Oyuncu listesinin tam olarak yüklenmesi bekleniyor...")
            wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")) >= 10)
        except TimeoutException:
            print("  Sayfada 10'dan az oyuncu bulundu, mevcut olanlar işlenecek.")
            pass

        player_rows = driver.find_elements(By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")
        
        if not player_rows:
            print("  Bu sayfada hiç oyuncu bulunamadı. Döngü sonlandırılıyor.")
            break
            
        print(f"  Bu sayfada {len(player_rows)} oyuncu satırı bulundu.")

        print("  Listenin sonuna kaydırılarak tüm oyuncuların görünür olması sağlanıyor...")
        driver.execute_script("arguments[0].scrollIntoView();", player_rows[-1])
        time.sleep(1) 

        page_data_batch = []
        for i, row in enumerate(player_rows):
            try:
                player_name_element = row.find_element(By.CSS_SELECTOR, "span.css-cfc8cy-TeamOrPlayerName")
                player_name = player_name_element.text.strip()
                
                stat_value_element = row.find_element(By.CSS_SELECTOR, "span.css-wo228r-StatValue span")
                stat_value = safe_float_extract(stat_value_element.text)
                
                rank_element = row.find_element(By.CSS_SELECTOR, "span.css-twq2db-Rank")
                rank = int(rank_element.text.strip())
                
                
                player_data = {
                    'Rank': rank,
                    'Player': player_name,
                    'Clean_Sheets': stat_value
                }
                page_data_batch.append(player_data)
                
            except Exception as e:
                print(f"  Hata: Bu sayfadaki {i+1}. oyuncu için veri çekilemedi. Hata: {e}")
                continue

        if page_data_batch:
            all_players_data.extend(page_data_batch)
            for p_data in page_data_batch:
                 
                 print(f"  ({p_data['Rank']}) {p_data['Player']} | Clean Sheets: {p_data['Clean_Sheets']}")
            print(f"  Sayfa {current_page} başarıyla işlendi ve {len(page_data_batch)} oyuncu eklendi.")

        if page_num < PAGES_TO_SCRAPE - 1:
            print(f"\n{current_page}. sayfadan {current_page + 1}. sayfaya geçiliyor...")
            
            try:
                next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Sonraki')]")))
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", next_button)
                print(f"  Başarıyla {current_page + 1}. sayfaya geçildi.")
                time.sleep(3)
                
            except Exception as e:
                print(f"  'Sonraki' butonu bulunamadı veya tıklanamadı. İşlem sonlandırılıyor. Hata: {e}")
                break

finally:
    driver.quit()
    print("\nSelenium WebDriver kapatıldı.")

if all_players_data:
    df = pd.DataFrame(all_players_data)
    
    df = df[['Rank', 'Player', 'Clean_Sheets']]
    
    df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')
    print(f"\nVeriler başarıyla '{OUTPUT_CSV_FILE}' dosyasına kaydedildi.")
    print(f"Toplam {len(df)} oyuncu verisi toplandı.")
    print("\n--- İlk 5 Satır ---")
    print(df.head())
    print("\n--- Son 5 Satır ---")
    print(df.tail())
else:
    print("\nHiçbir veri toplanamadığı için CSV dosyası oluşturulmadı.")

Selenium WebDriver başlatıldı.
Hedef URL'ye gidiliyor: https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/clean_sheet/premier-league
Çerez onay penceresi bekleniyor...
Çerez onay penceresi bulunamadı, devam ediliyor.

--- Sayfa 1 işleniyor ---
  Oyuncu listesinin tam olarak yüklenmesi bekleniyor...
  Sayfada 10'dan az oyuncu bulundu, mevcut olanlar işlenecek.
  Bu sayfada 9 oyuncu satırı bulundu.
  Listenin sonuna kaydırılarak tüm oyuncuların görünür olması sağlanıyor...
  (1) David Raya | Clean Sheets: 13.0
  (2) Matz Sels | Clean Sheets: 13.0
  (3) Jordan Pickford | Clean Sheets: 12.0
  (4) Dean Henderson | Clean Sheets: 11.0
  (5) Ederson Moraes | Clean Sheets: 10.0
  (6) Robert Sánchez | Clean Sheets: 10.0
  (7) Alisson Becker | Clean Sheets: 9.0
  (8) Andre Onana | Clean Sheets: 9.0
  (9) Nick Pope | Clean Sheets: 8.0
  Sayfa 1 başarıyla işlendi ve 9 oyuncu eklendi.

1. sayfadan 2. sayfaya geçiliyor...
  Başarıyla 2. sayfaya geçildi.

--- Sayfa 2 işleniyor ---
  Oyunc

## Kurtarış Yüzdesi

In [64]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import re


STATS_URL = "https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/_save_percentage/premier-league"
OUTPUT_CSV_FILE = "save_percentage_stats.csv"
PAGES_TO_SCRAPE = 3


options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument("log-level=3")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

print("Selenium WebDriver başlatıldı.")

all_players_data = []

def safe_float_extract(text):
    """Metinden ondalık sayısal değerleri güvenli bir şekilde çıkarır"""
    if not text:
        return 0.0
    
    text = text.replace(',', '.')
    numbers = re.findall(r'\d+\.?\d*', text.strip())
    if numbers:
        return float(numbers[0])
    return 0.0

try:
    driver.get(STATS_URL)
    print(f"Hedef URL'ye gidiliyor: {STATS_URL}")
    
    wait = WebDriverWait(driver, 15)

    try:
        print("Çerez onay penceresi bekleniyor...")
        accept_cookies_button = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_cookies_button.click()
        print("Çerezler kabul edildi.")
        time.sleep(2)
    except TimeoutException:
        print("Çerez onay penceresi bulunamadı, devam ediliyor.")

   
    for page_num in range(PAGES_TO_SCRAPE):
        current_page = page_num + 1
        print(f"\n--- Sayfa {current_page} işleniyor ---")

        try:
            print("  Oyuncu listesinin tam olarak yüklenmesi bekleniyor...")
            wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")) >= 10)
        except TimeoutException:
            print("  Sayfada 10'dan az oyuncu bulundu, mevcut olanlar işlenecek.")
            pass

        player_rows = driver.find_elements(By.CSS_SELECTOR, "div.css-1u3ja2l-LeagueSeasonStatsTableRowCSS")
        
        if not player_rows:
            print("  Bu sayfada hiç oyuncu bulunamadı. Döngü sonlandırılıyor.")
            break
            
        print(f"  Bu sayfada {len(player_rows)} oyuncu satırı bulundu.")

        print("  Listenin sonuna kaydırılarak tüm oyuncuların görünür olması sağlanıyor...")
        driver.execute_script("arguments[0].scrollIntoView();", player_rows[-1])
        time.sleep(1) 

        page_data_batch = []
        for i, row in enumerate(player_rows):
            try:
                player_name_element = row.find_element(By.CSS_SELECTOR, "span.css-cfc8cy-TeamOrPlayerName")
                player_name = player_name_element.text.strip()
                
                stat_value_element = row.find_element(By.CSS_SELECTOR, "span.css-wo228r-StatValue span")
                stat_value = safe_float_extract(stat_value_element.text)
                
                rank_element = row.find_element(By.CSS_SELECTOR, "span.css-twq2db-Rank")
                rank = int(rank_element.text.strip())
                
                
                player_data = {
                    'Rank': rank,
                    'Player': player_name,
                    'Save_Percentage': stat_value
                }
                page_data_batch.append(player_data)
                
            except Exception as e:
                print(f"  Hata: Bu sayfadaki {i+1}. oyuncu için veri çekilemedi. Hata: {e}")
                continue

        if page_data_batch:
            all_players_data.extend(page_data_batch)
            for p_data in page_data_batch:
                
                 print(f"  ({p_data['Rank']}) {p_data['Player']} | Save Percentage: {p_data['Save_Percentage']}")
            print(f"  Sayfa {current_page} başarıyla işlendi ve {len(page_data_batch)} oyuncu eklendi.")

        if page_num < PAGES_TO_SCRAPE - 1:
            print(f"\n{current_page}. sayfadan {current_page + 1}. sayfaya geçiliyor...")
            
            try:
                next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Sonraki')]")))
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", next_button)
                print(f"  Başarıyla {current_page + 1}. sayfaya geçildi.")
                time.sleep(3)
                
            except Exception as e:
                print(f"  'Sonraki' butonu bulunamadı veya tıklanamadı. İşlem sonlandırılıyor. Hata: {e}")
                break

finally:
    driver.quit()
    print("\nSelenium WebDriver kapatıldı.")

if all_players_data:
    df = pd.DataFrame(all_players_data)
    
    df = df[['Rank', 'Player', 'Save_Percentage']]
    
    df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')
    print(f"\nVeriler başarıyla '{OUTPUT_CSV_FILE}' dosyasına kaydedildi.")
    print(f"Toplam {len(df)} oyuncu verisi toplandı.")
    print("\n--- İlk 5 Satır ---")
    print(df.head())
    print("\n--- Son 5 Satır ---")
    print(df.tail())
else:
    print("\nHiçbir veri toplanamadığı için CSV dosyası oluşturulmadı.")

Selenium WebDriver başlatıldı.
Hedef URL'ye gidiliyor: https://www.fotmob.com/tr/leagues/47/stats/season/23685/players/_save_percentage/premier-league
Çerez onay penceresi bekleniyor...
Çerez onay penceresi bulunamadı, devam ediliyor.

--- Sayfa 1 işleniyor ---
  Oyuncu listesinin tam olarak yüklenmesi bekleniyor...
  Sayfada 10'dan az oyuncu bulundu, mevcut olanlar işlenecek.
  Bu sayfada 9 oyuncu satırı bulundu.
  Listenin sonuna kaydırılarak tüm oyuncuların görünür olması sağlanıyor...
  (1) Mark Flekken | Save Percentage: 73.6
  (2) Robert Sánchez | Save Percentage: 73.6
  (3) Jordan Pickford | Save Percentage: 73.5
  (4) Alisson Becker | Save Percentage: 72.9
  (5) Matz Sels | Save Percentage: 72.3
  (6) David Raya | Save Percentage: 71.7
  (7) Nick Pope | Save Percentage: 71.5
  (8) Kepa Arrizabalaga | Save Percentage: 71.3
  (9) Emiliano Martinez | Save Percentage: 69.0
  Sayfa 1 başarıyla işlendi ve 9 oyuncu eklendi.

1. sayfadan 2. sayfaya geçiliyor...
  Başarıyla 2. sayfaya g